In [2]:
### Import modules
from mpi4py import MPI
import gmsh
from dolfinx.io import gmshio
import dolfinx.mesh as msh

import numpy as np

from dolfinx.fem import Function, FunctionSpace, assemble, form, petsc, Constant
from ufl import (TestFunction, TrialFunction, TrialFunctions,
                 dx, grad, inner, Measure, FacetNormal, CellNormal)
from dolfinx import plot
import petsc4py
from petsc4py import PETSc
import pyvista

if False:
    import pyvista as pv
    pv.start_xvfb()
    pv.set_jupyter_backend('client')
    sphere = pv.Sphere()
    
    # short example
    sphere.plot(jupyter_backend='client')

### Define geometry
model_name = 'test'
gmsh.initialize()
comm = MPI.COMM_WORLD
model_rank = 0
model = gmsh.model()
gmsh.model.add(model_name)

radius = 0.3
lc = 2e-2
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc)
p2 = gmsh.model.geo.addPoint(1, 0, 0, lc)
p3 = gmsh.model.geo.addPoint(1, 1, 0, lc)
p4 = gmsh.model.geo.addPoint(0, 1, 0, lc)
p5 = gmsh.model.geo.addPoint(0, radius, 0, lc)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
l5 = gmsh.model.geo.addLine(p5, p1)

cl1 = [l1, l2, l3, l4, l5]
s1 = gmsh.model.geo.addPlaneSurface([gmsh.model.geo.addCurveLoop(cl1)])
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(2, [s1], tag=1)
gmsh.model.addPhysicalGroup(1, [l5], tag=1)  # Neumann
gmsh.model.addPhysicalGroup(1, [l2, l3], tag=3)

gmsh.model.mesh.generate(2)
final_mesh, mesh_tags, mesh_bc_tags = gmshio.model_to_mesh(model, comm, model_rank)
#gmsh.model.geo.synchronize()
gmsh.finalize()

tdim = final_mesh.topology.dim
fdim = tdim - 1

xref = [0, 0, 0]

submesh, entity_map = msh.create_submesh(final_mesh, fdim, mesh_bc_tags.find(3))[0:2]

tdim = final_mesh.topology.dim
fdim = tdim - 1
submesh_tdim = submesh.topology.dim
submesh_fdim = submesh_tdim - 1

mesh_facet_imap = final_mesh.topology.index_map(fdim)
mesh_num_facets = mesh_facet_imap.size_local + mesh_facet_imap.num_ghosts
entity_maps_mesh = {submesh: [entity_map.index(entity)
                                if entity in entity_map else -1
                                for entity in range(mesh_num_facets)]}



### 
rho0 = 1.21
c0 = 343.8
source = 1
deg = 2
# Define the function spaces
###########################
P = FunctionSpace(final_mesh, ("Lagrange", deg))
Q = FunctionSpace(submesh, ("Lagrange", deg-1))


if False:
    pyvista.start_xvfb()
    u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(Q)
    u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
    u_plotter = pyvista.Plotter()
    u_plotter.add_mesh(u_grid, show_edges=True)
    u_plotter.view_xy()
    if not pyvista.OFF_SCREEN:
        u_plotter.show(jupyter_backend='client')
# Define variational problem
############################
p, q = TrialFunction(P), TrialFunction(Q)
v, u = TestFunction(P), TestFunction(Q)

fx = Function(Q)
fx.interpolate(lambda x: 1/np.sqrt((x[0]-xref[0])**2 + (x[1]-xref[1])**2 + (x[2]-xref[2])**2)) # fx represents 1/r which is the inverse of the distance from the center of the piston

# Create measure for integral over domain of boundary
dx = Measure("dx", domain=final_mesh, subdomain_data=mesh_tags)
ds = Measure("ds", domain=final_mesh, subdomain_data=mesh_bc_tags)
dx1 = Measure("dx", domain=submesh)
n = FacetNormal(final_mesh) # Normal to the boundaries
sn = CellNormal(submesh)


freq = 1000
k0 = 2*np.pi*freq/c0

#Fx = (2*1j*k0+4*fx)
#Hx = (2*fx**2-k0**2+4*1j*k0*fx)

k = inner(grad(p), grad(v)) * dx # scalar product of the gradiants
m = inner(p, v) * dx # dx represents the integral over the whole considered domain, here it is the whole acoustic domain
c = inner(q, v)*ds(3) # ds represents the integral over a boundary, here the '3' one, which is the boundary where ABC is applied
   
t   = inner(fx*p, u)*ds(3)
dp  = inner(grad(p), n) # dp/dn = grad(p) * n
#ddp = inner(grad(dp), n) # d^2p/dn^2 = grad(dp/dn) * n = grad(grad(p) * n) * n
#dt  = inner(dp, u)*ds(3)

dt  = inner(dp, u)*ds(3)

dq  = inner(grad(q), sn)
de  = inner(dq, u)*ds(3)
e = inner(fx*q, u)*ds(3) # q and u are defined in Q which is the function space defined on the boundary where ABC is applied as a whole surface domain

a_00 = form(k - k0**2*m) # create square matrix
a_01 = form(-c, entity_maps=entity_maps_mesh) # create rectangular matrix
a_10 = form(t, entity_maps=entity_maps_mesh)
a_11 = form(e + de, entity_maps=entity_maps_mesh)

a = [[a_00, a_01],
    [a_10, a_11]]
A = petsc.assemble_matrix_block(a)
A.assemble()

L = form([inner(source, v) * ds(1), inner(Constant(submesh, PETSc.ScalarType(0)), u) * dx1])
b = petsc.assemble_vector_block(L, a)

ksp = PETSc.KSP().create(final_mesh.comm)
ksp.setOperators(A)
ksp.setType("gmres") # Solver type 
ksp.getPC().setType("lu") # Preconditionner type
ksp.getPC().setFactorSolverType("mumps")

X = A.createVecLeft()
ksp.solve(b, X)

Psol1, Qsol1 = Function(P), Function(Q)
offset = P.dofmap.index_map.size_local * P.dofmap.index_map_bs
Psol1.x.array[:offset] = X.array_r[:offset]
Qsol1.x.array[:(len(X.array_r) - offset)] = X.array_r[offset:]

Pav1 = petsc.assemble.assemble_scalar(form(Psol1*ds(1)))
ksp.destroy()
X.destroy()
A.destroy()
b.destroy()

print(Pav1)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.000330208s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0585033s, CPU 0.058732s)
Info    : 3017 nodes 6037 elements


ValueError: Cell normal undefined for tdim 1, gdim 3